## 利用 Ollama API 整合 OpenAI Python 函式庫
Ollama 提供了與 OpenAI API 部分功能的相容，協助現有應用程式連接到 Ollama。這篇教學文件將引導你如何利用 OpenAI 函式庫與 Ollama API 整合，快速切換不同家的大型語言模型（LLM）服務。若要了解如何安裝 Ollama，請參考前一篇文章。利用 OpenAI 庫的優點在於，它相容市面上大多數 LLM 服務，只需替換 API URL，即可快速整合不同廠商的服務。

## 1. 環境設定與客戶端初始化
首先，我們需要安裝 OpenAI 函式庫（可透過 pip 安裝），接著透過下列程式碼初始化客戶端，並指定 API 伺服器的 URL 與金鑰。  
以下範例中使用的是本機伺服器（例如：http://localhost:11434/v1/ ），金鑰雖必填，但此處實際上不會使用。

In [3]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',  # 設定 API 伺服器的 URL，這裡使用本機端點
    api_key='ollama',                      # API 金鑰（此例中為 'ollama'）
)

## 2. Chat Completion API（對話生成）
### 2.1 純文字聊天

In [4]:
# 使用 chat API 建立一個聊天請求
chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',  # 設定角色為「使用者」
            'content': '法國的首都在哪?',  # 提供要詢問的問題（提示詞）
        }
    ],
    model='gemma2:9b',  # 指定模型
)

# 取得模型回應的內容
print(chat_completion.choices[0].message.content)

法國的首都是 **巴黎**。 🗼  



### 2.2 文字與影像混合輸入的聊天

Ollama 的 API 除了支援純文字外，還可支援影像輸入。以下範例展示如何同時傳送文字與以 base64 編碼的影像資料：


In [38]:
import openai
import base64

# 讀取本機圖片並轉為 Base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 圖片路徑
image_path = "./image.jpg"
base64_image = encode_image(image_path)

In [39]:
response = client.chat.completions.create(
    model="llama3.2-vision:11b",  # 確保使用支援 Vision 的模型
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "這張圖片裡有什麼?"},
                {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}
            ]
        }
    ],
    max_tokens=300,
)

# 取得回應
print(response.choices[0].message.content)

可以在這個藍色圓圈內看到有一些暗棕色的文字，然而由於字體過小和顏色不明顯，因此很難辨別。


## 3. Completion API(單次問答)
若不需要多輪對話，也可以直接使用單次問答 API。以下範例以 prompt 提示「美國首都在哪裡?」並使用 `gemma2:9b` 模型取得提問結果：

In [47]:
completion = client.completions.create(
    model="gemma2:9b", # 指定模型
    prompt="美國首都在哪裡?", # 提供要詢問的問題（提示詞）
)

# 取得模型回應的內容
print(completion.choices[0].text)

美國的首都是 **華盛頓哥倫比亞區**。 



## 4. 模型操作
### 4.1 列出所有可用模型

你可以使用下列程式碼列出目前 API 支援的所有模型：

In [52]:
from datetime import datetime

# 取得可用模型列表
list_completion = client.models.list()

# 轉換 Unix 時間戳
for model in list_completion.data:
    readable_time = datetime.utcfromtimestamp(model.created).strftime('%Y-%m-%d %H:%M:%S')
    print(f"模型 ID: {model.id}")
    print(f"建立時間: {readable_time}")
    print(f"類型: {model.object}")
    print(f"擁有者: {model.owned_by}")
    print("-" * 30)


模型 ID: llama3.2-vision:11b
建立時間: 2025-02-19 12:21:06
類型: model
擁有者: library
------------------------------
模型 ID: gemma2:9b
建立時間: 2025-02-19 12:08:58
類型: model
擁有者: library
------------------------------


### 4.2 取得特定模型資訊
若想查詢某一特定模型的詳細資訊，可使用：

In [57]:
from datetime import datetime

model = client.models.retrieve("gemma2:9b")

print(f"模型 ID: {model.id}")
print(f"建立時間: {datetime.utcfromtimestamp(model.created):%Y-%m-%d %H:%M:%S}")
print(f"類型: {model.object}")
print(f"擁有者: {model.owned_by}")

模型 ID: gemma2:9b
建立時間: 2025-02-19 12:08:58
類型: model
擁有者: library


## 5. 產生 Embeddings

另一項常用的功能是利用 embeddings 將句子轉換為向量表示，這對於語意相似度、文本分類等應用非常有用。需要先下載 all-minilm:22m 模型，這個模型是輕量級的 sentence embedding 模型。

```sh
ollama pull all-minilm:22m
```



以下範例使用 `all-minilm` 模型來為兩個句子生成 embeddings：

In [18]:
# 產生 Embeddings
embeddings = client.embeddings.create(
    model="all-minilm:22m",  # 指定 embedding 模型
    input=["why is the sky blue?", "why is the grass green?"],  # 要轉換成向量的文本
)

# 取出第一個句子的向量
embedding_1 = embeddings.data[0].embedding
# 取出第二個句子的向量
embedding_2 = embeddings.data[1].embedding

# 只印出前 5 維，避免太長
print("\n第一個句子的 Embedding（前 5 維）：")
print(embedding_1[:5])

print("\n第二個句子的 Embedding（前 5 維）：")
print(embedding_2[:5])


第一個句子的 Embedding（前 5 維）：
[0.010048831, -0.0017536068, 0.05003938, 0.04698401, 0.055051647]

第二個句子的 Embedding（前 5 維）：
[-0.009840123, 0.060384717, 0.025232576, -0.0062687513, 0.07272164]


### **預設使用的模型**
`all-minilm` 是 **MiniLM**（Minimal Language Model）的變體，專門用來產生輕量級的 **句子向量（sentence embeddings）**。  
這類模型主要來自 **Sentence-Transformers**，類似 **`all-MiniLM-L6-v2`** 這種模型，擅長計算句子之間的語意相似度，並且效能高、計算速度快。

### **這段程式碼的作用**
這行程式碼將兩個問題：
- `"why is the sky blue?"`
- `"why is the grass green?"`

轉換成 **向量表示**，並且可以用來計算語意相似度，例如：
- 如果兩個句子的意思很接近，則它們的向量之間的距離會很小（通常用 **餘弦相似度** 來計算）。
- 這樣可以應用在 **搜尋引擎、文本分類、推薦系統、問答系統等**。

In [19]:
# 計算語意相似度（Cosine Similarity）
import numpy as np

cosine_similarity = np.dot(embedding_1, embedding_2) / (
    np.linalg.norm(embedding_1) * np.linalg.norm(embedding_2)
)

print(f"\n兩個句子的語意相似度（Cosine Similarity）：{cosine_similarity:.4f}")


兩個句子的語意相似度（Cosine Similarity）：0.4734


## Reference
- [Ollama官方API文檔](https://github.com/ollama/ollama/blob/main/docs/api.md)
- [Ollama官方相容Openai API文檔](https://github.com/ollama/ollama/blob/main/docs/openai.md)